# Toxic Comment Classification Challenge
## Identify and classify toxic online comments

El problema conciste en crear un modelo el cual clasifique los textos en 6 categorias:
1. comment_text
2. toxic
3. severe_toxic
4. obscene
5. threat
6. insult
7. identity_hate

Para ello se dividira el proceso en las siguientes etapas:
1. [Preprocesamiento](#pre)
2. [Analisis estadístico](#eda)
3. [Modelado y Evaluación](#mod)

### Paqueterías
Aqui se incluyen las paqueterias necesarias.

In [3]:
import pandas as pd
import spacy
from gensim.models import Phrases
from funcs import entidades, quitarpuntuacion, bigramas
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

<a id='pre'></a>
## Preprocesamiento

El preprocesamiento conciste en limpriar los textos para poder despues . Primero cargamos los datos.

In [4]:
train = pd.read_csv('data/train.csv',index_col=0)
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['limpio'] = 1-train[label_cols].max(axis=1)
train['comment_text'].fillna("unknown", inplace=True)
test['comment_text'].fillna("unknown", inplace=True)

Observamos que contiene el archivo de train

In [7]:
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


Observemos el texto del segundo renglón.

In [8]:
train.iloc[1].comment_text

'"\n\n Please do not vandalize pages, as you did with this edit to W. S. Merwin. If you continue to do so, you will be blocked from editing.    "'

Lo que haremos en el pre-procesamiento va a ser:
- Quitar saltos de linea (\n)
- Quitar puntuación y simbolos (.:;,!"#$%&/()=?)
- Separar las palabras y crear n-gramas, por ejemplo en vez de separar en tres palabras "W.", "S." y "Merwin", que sea una sola palabra, aun que en realidad se podrian borrar estas palabras, ya que no son importantes para la clasificación.
- Tokeniza, que es separar las palabras. 

In [5]:
nlp = spacy.load('en')
train = quitarpuntuacion(train)
docs = train.comment_text

La parte de tokenizar y crear los bigramas se hacen con funciones definidas en el archivo funcs.py que esta en la misma carpeta

In [ ]:
%%time
docs = entidades(docs,nlp)
docs = bigramas(docs)

Dado que este ultimo procesa lleva una tiempo considerable se guarda.

In [ ]:
pickle.dump( docs, open( "save.p", "wb" ) )

docs = pickle.load( open( "save.p", "rb" ) )


<a id='eda'></a>
## Analisis estadistico

In [1]:
# TODO

<a id='mod'></a>
## Modelo y evaluación
Debido a que los datos a predecir son categorias y no son excluyentes, es decir un documento puede pertenecer a dos o más categorias se debe hacer un modelo para cada categoria. Para un primer ejercicio se usaran los siguientes modelos:
- Naive Bayes multinomial
- Regresión Logistica
- Stochastic Gradient Descent

Antes de configurar los modelos los datos se tienen que veectorizar, i.e. acomodar en una matriz numérica, por lo se se usa Tf-idf.

In [6]:
train['texto_limpio'] = docs
#train['texto_limpio'] = train['texto_limpio'].apply(lambda x: ' '.join(x))
del train['comment_text']
del train['limpio']

Se divide los datos en datos de entrenamiento y datos de prueba, para despues vectorizarlos

In [7]:
#%%Dividir

X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,6], train.iloc[:,0:6],test_size=0.2)

#%%
vectorizer = TfidfVectorizer(stop_words = 'english',\
                             vocabulary=None,\
                             analyzer='word',\
                             lowercase=True,\
                             ngram_range=(1, 1),\
                             max_df=1.0,\
                             min_df=1)

tfidf_train = vectorizer.fit_transform(X_train)
tfidf_test = vectorizer.transform(X_test)

### Regresión logística

In [8]:
reglog = LogisticRegression()
presicion_reglog = dict()

for label in label_cols:
    y = y_train[label]
    reglog.fit(tfidf_train, y)
    y_hat = reglog.predict(tfidf_test)
    presicion_reglog[label] = accuracy_score(y_test[label], y_hat)

### Naive Bayes multinomial

In [9]:
naive_bayes = MultinomialNB()
presicion_naive_bayes = dict()

for label in label_cols:
    y = y_train[label]
    naive_bayes.fit(tfidf_train, y)
    y_hat = naive_bayes.predict(tfidf_test)
    presicion_naive_bayes[label] = accuracy_score(y_test[label], y_hat)

### Stochastic Gradient Descent
> Nota: Independiente al resultado de este modelo, para la competencia de kaggle no funciona ya que los datos que tienes que regresar es una probabilidad, y SGD regresa si pertenece o no a la categoría.

In [10]:
SGDC = SGDClassifier()
presicion_SGDC = dict()

for label in label_cols:
    y = y_train[label]
    SGDC.fit(tfidf_train, y)
    y_hat = SGDC.predict(tfidf_test)
    presicion_SGDC[label] = accuracy_score(y_test[label], y_hat)

/home/pepe/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### Evaluación

In [11]:
print(sum(list(presicion_reglog.values()))/6)
print(sum(list(presicion_naive_bayes.values()))/6)
print(sum(list(presicion_SGDC.values()))/6)

0.979816178391
0.966624889028
0.976844743851


Por lo que elgimos la regresión logistica

#### Submission
Por ultimo creamos el archivo para subirlo a la competencia:

In [12]:
tfidf_train_c =vectorizer.fit_transform(train['texto_limpio'])
tfidf_sub = vectorizer.transform(test['comment_text'])

for label in label_cols:
    y = train[label]
    reglog.fit(tfidf_train_c, y)
    test[label] = reglog.predict_proba(tfidf_sub)[:,1]
#%%

#%%
subm = test
del subm["comment_text"]
#%%
subm.to_csv('subs/submission5.csv',index=False)

## ¿Qué sigue?
Para mejorar el modelo, las ideas que se podrian hacer son:
- Mejorar el preprocesamiento: aumentar los bi-gramas a n-gramas, filtrar palabras comunes aunque no sean stop words
- Mejorar la vectorización: Reducir el tamaño, eliminando palabras
- Probar más modelos: Usar redes nuronales, otros clasificadores(e.g. NB-SVM).
- Ver la calidad del modelo logistico: Checar curva ROC, Recall, Presicion, etc.